In [2]:
import os
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import cpi
import pymongo
from tqdm import tqdm
from scipy import stats
import math

In [3]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient["imdb"]
imdb = db['imdb_cleaned']

In [5]:
num_movies = imdb.count_documents({})
num_movies

8181

In [6]:
rev_arr = []
for movie in imdb.find():
    rev = movie['cleaned_Revenue']
    rev_arr.append(rev) 

In [10]:
import numpy as np

In [11]:
rev_arr = np.array(rev_arr)

In [12]:
stats.describe(rev_arr)

DescribeResult(nobs=8181, minmax=(38, 3550221370), mean=100747045.3002078, variance=4.423010334815394e+16, skewness=5.235542127662483, kurtosis=45.489504310842435)

In [13]:
def findIMDbMovie(imdb_id):
    for x in imdb.find({'imdb_id': imdb_id}):
        return x
    return None

In [14]:
# findIMDbMovie('tt0371746')

In [15]:
bucket_size = 50000000 # 50 million
buckets = {}
curr = 0
while(curr <= rev_arr.max()):
    buckets[curr] = 0
    curr += bucket_size 

In [16]:
for movie in imdb.find():
    rev = movie['cleaned_Revenue']
    lo = int(np.floor(rev/bucket_size)*bucket_size)
    buckets[lo] += 1

In [17]:
million = 10**6
million

1000000

In [20]:
xrr = []
yrr = []

cnt = 0
for lo in buckets:
    hi = lo + bucket_size
    cnt += buckets[lo]
    xrr.append('{}M - {}M'.format(lo/million, hi/million, cnt, cnt/num_movies))
    yrr.append(buckets[lo])

    print('low range {} -- num movies {} -- total {}({}%) of {} movies'.format(lo, buckets[lo], cnt, cnt*100/num_movies, num_movies))

low range 0 -- num movies 5082 -- total 5082(62.11954528786212%) of 8181 movies
low range 50000000 -- num movies 1063 -- total 6145(75.11306686224178%) of 8181 movies
low range 100000000 -- num movies 527 -- total 6672(81.55482214888156%) of 8181 movies
low range 150000000 -- num movies 351 -- total 7023(85.84525119178585%) of 8181 movies
low range 200000000 -- num movies 248 -- total 7271(88.87666544432221%) of 8181 movies
low range 250000000 -- num movies 177 -- total 7448(91.04021513262437%) of 8181 movies
low range 300000000 -- num movies 131 -- total 7579(92.64148637085931%) of 8181 movies
low range 350000000 -- num movies 109 -- total 7688(93.9738418286273%) of 8181 movies
low range 400000000 -- num movies 73 -- total 7761(94.86615328199487%) of 8181 movies
low range 450000000 -- num movies 60 -- total 7821(95.5995599559956%) of 8181 movies
low range 500000000 -- num movies 49 -- total 7870(96.19850873976287%) of 8181 movies
low range 550000000 -- num movies 44 -- total 7914(96.7

In [29]:
fig = go.Figure([go.Bar(x=xrr, y=yrr)])
fig.show()